# 4-1.Modeling

In [2]:
import os
import pathlib
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
from haversine import haversine
import seaborn as sns
import json
from shapely.geometry import Point
sns.set()

In [3]:
#Font
import matplotlib
from matplotlib import font_manager
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")
matplotlib.rcParams['axes.unicode_minus']=False

font_fname = '/opt/app-root/lib/python3.6/site-packages/matplotlib/mpl-data/fonts/ttf/malgun.ttf'
font_family = font_manager.FontProperties(fname=font_fname).get_name()
plt.rcParams["font.family"] = font_family

In [4]:
path = pathlib.Path('./data')

In [5]:
주정차단속 = pd.read_csv(os.path.join(path,'1.오산시_주정차단속(2018~2020).csv'),encoding="utf-8")
어린이교통사고_격자 = gpd.read_file(os.path.join(path,'2.오산시_어린이교통사고_격자.geojson'))
차량등록현황_격자 = gpd.read_file(os.path.join(path,'3.오산시_차량등록현황_격자.geojson'))
거주인구격자_총인구 = gpd.read_file(os.path.join(path,'4.오산시_연령별_거주인구격자(총인구).geojson'))
거주인구격자_유소년 = gpd.read_file(os.path.join(path,'5.오산시_연령별_거주인구격자(유소년).geojson'))
거주인구격자_생산가능 = gpd.read_file(os.path.join(path,'6.오산시_연령별_거주인구격자(생산가능인구).geojson'))
거주인구격자_고령 = gpd.read_file(os.path.join(path,'7.오산시_연령별_거주인구격자(고령).geojson'))
유동인구 = pd.read_csv(os.path.join(path,'8.오산시_유동인구(2019).csv'),encoding="utf-8")
어린이보호구역 = pd.read_csv(os.path.join(path,'9.오산시_어린이보호구역.csv'),encoding="utf-8")
학교위치정보 = pd.read_csv(os.path.join(path,'10.오산시_학교위치정보.csv'),encoding="utf-8")
초등학교_통학구 = gpd.read_file(os.path.join(path,'11.오산시_초등학교_통학구.geojson'))
중학교_학군 = gpd.read_file(os.path.join(path,'12.오산시_중학교_학군.geojson'))
어린이집_유치원현황 = pd.read_csv(os.path.join(path,'13.오산시_어린이집_유치원현황.csv'),encoding="utf-8")
기상데이터 = pd.read_csv(os.path.join(path,'14.오산시_기상데이터(2010~2019).csv'),encoding="utf-8")
무인교통단속카메라 = pd.read_csv(os.path.join(path,'15.오산시_무인교통단속카메라.csv'),encoding="utf-8")
도로안전표지 = pd.read_csv(os.path.join(path,'16.오산시_도로안전표지표준데이터.csv'),encoding="utf-8")
횡단보도 = gpd.read_file(os.path.join(path,'17.오산시_횡단보도.geojson'))
과속방지턱 = pd.read_csv(os.path.join(path,'18.오산시_과속방지턱표준데이터.csv'),encoding="utf-8")
신호등 = gpd.read_file(os.path.join(path,'19.오산시_신호등.geojson'))
CCTV설치현황 = pd.read_csv(os.path.join(path,'20.오산시_CCTV설치현황.csv'),encoding="utf-8")
인도 = gpd.read_file(os.path.join(path,'21.오산시_인도.geojson'))
버스정류장 = pd.read_csv(os.path.join(path,'22.오산시_버스정류장.csv'),encoding="utf-8")
상세도로망 = gpd.read_file(os.path.join(path,'23.오산시_상세도로망_LV6.geojson'))
시간대별_추정교통량 = pd.read_csv(os.path.join(path,'24.평일_전일,시간대별_오산시_추정교통량_Level6.csv'),encoding="utf-8")
혼잡빈도강도 = pd.read_csv(os.path.join(path,'25.평일_전일_오산시_혼잡빈도강도_Level6.csv'),encoding="utf-8")
혼잡시간강도 = pd.read_csv(os.path.join(path,'26.평일_전일_오산시_혼잡시간강도_Level6.csv'),encoding="utf-8")
도로명주소_건물 = gpd.read_file(os.path.join(path,'27.오산시_도로명주소_건물.geojson'))
건물연면적_격자 = gpd.read_file(os.path.join(path,'28.오산시_건물연면적_격자.geojson'))
체육시설현황 = pd.read_csv(os.path.join(path,'29.오산시_체육시설현황.csv'),encoding="utf-8")
학원_및_교습소_현황 = pd.read_csv(os.path.join(path,'30.오산시_학원_및_교습소_현황.csv'),encoding="utf-8")
법정경계_시군구 = gpd.read_file(os.path.join(path,'31.오산시_법정경계(시군구).geojson'))
행정경계_읍면동 = gpd.read_file(os.path.join(path,'32.오산시_행정경계(읍면동).geojson'))
법정경계_읍면동 = gpd.read_file(os.path.join(path,'33.오산시_법정경계(읍면동).geojson'))
지적도 = gpd.read_file(os.path.join(path,'34.오산시_지적도.geojson'))

# [과제 1] 모델링을 위한 feature column 생성

__어린이 교통사고 격자 데이터를 기본으로 칼럼을 누적__

In [6]:
data = 어린이교통사고_격자[어린이교통사고_격자.accident_cnt>0].reset_index().copy()
data = data.drop(["index"],axis=1)

In [7]:
data.head(3)

,gid,accident_cnt,geometry
0,다사561083,1,"MULTIPOLYGON (((127.00549 37.17243, 127.00549 ..."
1,다사564096,1,"MULTIPOLYGON (((127.00880 37.18416, 127.00879 ..."
2,다사566102,1,"MULTIPOLYGON (((127.01101 37.18958, 127.01101 ..."


> __사고 격자 내 평균 유동인구 (col_name = 유동인구_sum)__

In [ ]:
def ppl_flow(geo_obj):
    
    global 유동인구
    
    ret_cnt = 0
    
    for (lon,lat,cnt) in 유동인구.iloc:
        current_point = Point(lon,lat)
        if geo_obj.contains(current_point):
            ret_cnt+=cnt
    return ret_cnt

data['유동인구_sum'] = data['geometry'].apply(ppl_flow)

> __사고 격자 주변(100m , 200m) 유소년 인구 (col_name = 유소년_count_100 , 유소년_count_200)__

In [ ]:
def young_living_nearby(geo_obj,rad):
    
    global 거주인구격자_유소년
    
    ret_cnt = 0
    y,x = geo_obj.centroid.coords[0]
    
    for (_,cnt,square) in 거주인구격자_유소년.iloc:
        c_y,c_x = square.centroid.coords[0]
        
        if haversine((x,y),(c_x,c_y),unit="m") <= rad:
            ret_cnt+=cnt
            
    return ret_cnt

data['유소년인구_count_100'] = data['geometry'].apply(lambda x:young_living_nearby(x,100))
data['유소년인구_count_200'] = data['geometry'].apply(lambda x:young_living_nearby(x,200))

> __사고 격자 주변(100m) 생산가능,고령,총 인구 (col_name = 생산가능인구_count_100 , 고령인구_count_100 , 총인구_count_100)__

In [ ]:
def other_living_nearby(geo_obj,other):
    
    ret_cnt = 0
    y,x = geo_obj.centroid.coords[0]
    
    for (_,cnt,square) in other.iloc:
        c_y,c_x = square.centroid.coords[0]
        
        if haversine((x,y),(c_x,c_y),unit="m") <= 100:
            ret_cnt+=cnt
            
    return ret_cnt

data['생산가능인구_count_100'] = data['geometry'].apply(lambda x:other_living_nearby(x,거주인구격자_생산가능))
data['고령인구_count_100'] = data['geometry'].apply(lambda x:other_living_nearby(x,거주인구격자_고령))
data['총인구_count_100'] = data['geometry'].apply(lambda x:other_living_nearby(x,거주인구격자_총인구))

> __사고 격자 주변(100m , 300m) 학원 수 (col_name = 학원수_count_100 , 학원수_count_300)__

In [ ]:
def hakwon_nearby(geo_obj,rad):
    
    global 학원_및_교습소_현황
    
    ret_cnt = 0
    y,x = geo_obj.centroid.coords[0]
    
    for (_1,_2,c_y,c_x) in 학원_및_교습소_현황.iloc:
        if haversine((x,y),(c_x,c_y),unit="m") <= rad:
            ret_cnt+=1
    
    return ret_cnt

data['학원수_count_100'] = data['geometry'].apply(lambda x : hakwon_nearby(x,100))
data['학원수_count_300'] = data['geometry'].apply(lambda x : hakwon_nearby(x,300))

> __사고 격자 소속 동 (col_name = 동_구분)__

In [ ]:
def dong_assigned(geo_obj):
    
    global 법정경계_읍면동
    
    point = geo_obj.centroid
    
    for line in 법정경계_읍면동.iloc:
        dong = line.geometry
        if dong.contains(point):
            return line.EMD_KOR_NM
    return np.nan

data["동_구분"] = data["geometry"].map(dong_assigned)
data = data.join(pd.dummies(data["동_구분"],drop_first=True,prefix="동")) # One-Hot-Encoding 칼럼 별도 추가

> __사고 격자 소속 초등학교 통학구 (col_name = 통학구_구분)__

In [ ]:
def tonghakgu(geo_obj):
    
    global 초등학교_통학구
    
    point = geo_obj.centroid
    
    for line in 초등학교_통학구.iloc:
        dong = line.geometry
        if dong.contains(point):
            return line["학구명"]
    return np.nan

data["통학구_구분"] = data["geometry"].map(tonghakgu)
data = data.join(pd.dummies(data["통학구_구분"],drop_first=True,prefix="통학구")) # One-Hot-Encoding 칼럼 별도 추가

> __사고 격자 내 신호등 수 (col_name = 신호등_sum)__

In [ ]:
def traffic_light(geo_obj):
    
    global 신호등

    cnt = 0
    for line in 신호등.iloc:
        _,obj = line
        if geo_obj.contains(obj):
            cnt+=1
    return cnt

data["신호등_sum"] = data["geometry"].map(traffic_light)

> __사고 격자 내 횡단보도 수 (col_name = 횡단보도_sum)__

In [ ]:
def cross_walk(geo_obj):
    
    global 횡단보도

    cnt = 0
    for line in 횡단보도.iloc:
        _,obj = line
        if geo_obj.contains(obj):
            cnt+=1
    return cnt

data["횡단보도_sum"] = data["geometry"].map(cross_walk)

> __사고 격자 주변(200m) 차량 등록 수 (col_name = 차량등록_count_200)__

In [ ]:
def car_registered(geo_obj):
    
    global 차량등록현황_격자
    
    y,x = geo_obj.centroid.coords[0]
    cnt=0
    for line in 차량등록현황_격자[차량등록현황_격자.car_cnt>0].iloc:
        _,val,obj_ = line

        if haversine((x,y),(obj_.centroid.coords[0][::-1]),unit="m")<=200:
            cnt+=val
    return cnt

data["차량등록_count_200"] = data["geometry"].map(car_registered)

> __사고 격자 주변(300m) 주정차 단속 건수 (col_name = 주정차_count_300)__

In [ ]:
def joojeong(geo_obj):
    
    global 주정차단속
    
    cnt=0
    for line in 주정차단속.iloc:
        y,x = line[-3],line[-2]

        if haversine((x,y),(geo_obj.centroid.coords[0][::-1]),unit="m")<=300:
            cnt+=1
    return cnt

data["주정차_count_300"] = data["geometry"].map(joojeong)